In [4]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 29.69 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [5]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [6]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Retrieving notices: done
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - compilers=1.10
    - go=1.24.5
    - jq=1.8.1
    - make=4.4.1
    - zlib=1.3.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _go_select-2.2.0           |            nocgo           5 KB  conda-forge
    binutils-2.43              |       h4852527_5          34 KB  conda-forge
    binutils_linux-64-2.43     |       h4852527_4          35 KB  conda-forge
    c-compiler-1.10.0          |       h2b85faf_0           7 KB  conda-forge
    ca-certificates-2025.7.14  |       hbd8a1cb_0         152 KB  conda-forge

In [7]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [8]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [9]:
# Installation d'obitools V4
cd obitools4
make
cd ..

mkdir -p build
Update version.go : pkg/obioptions/version.go to Release 4.4.0 (996ec69)
- Building obitool obiannotate...Done.
- Building obitool obiclean...Done.
- Building obitool obicleandb...Done.
- Building obitool obicomplement...Done.
- Building obitool obiconsensus...Done.
- Building obitool obiconvert...Done.
- Building obitool obicount...Done.
- Building obitool obicsv...Done.
- Building obitool obidemerge...Done.
- Building obitool obidistribute...Done.
- Building obitool obigrep...Done.
- Building obitool obijoin...Done.
- Building obitool obikmermatch...Done.
- Building obitool obikmersimcount...Done.
- Building obitool obilandmark...Done.
- Building obitool obimatrix...Done.
- Building obitool obimicrosat...Done.
- Building obitool obimultiplex...Done.
- Building obitool obipairing...Done.
- Building obitool obipcr...Done.
- Building obitool obireffamidx...Done.
- Building obitool obirefidx...Done.
- Building obitool obiscript...Done.
- Building obitool obisplit...Done.
-

In [ ]:
## FORMATAGE DES DONNES / ESPACES RETIRE
shopt -s nullglob
for f in galaxy_inputs/inputfile/*.{fastq*,fasta}; do
  [ -e "$f" ] || continue
  dir=$(dirname -- "$f")
  base=$(basename -- "$f")
  # on enlève les espaces
  newbase=${base// /_}
  newpath="$dir/$newbase"
  [ "$newpath" = "$f" ] && continue
  echo "Renomme: $f -> $newpath"
  mv -- "$f" "$newpath"
done

In [ ]:
# Recherche des chemins des inputs et formatage de l'ouput 
jq '.inputfile[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_fixed.json
json_path="galaxy_inputs/galaxy_inputs_fixed.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')


echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"

format=""

Input file path: galaxy_inputs/inputfile/obiuniqondata24.fasta
Nom de fichier (sans chemin, sans extension) : obiuniqondata24
Extension détectée : fasta


In [ ]:
#### Variables obigrep
listarg=(seq def ident attrib hasattrib predicat)

for i in "${listarg[@]}"; do
    if [[ -d "galaxy_inputs/$i" ]]; then
        value=$(head -n 1 "galaxy_inputs/$i"/*.txt)  # lit la 1ère ligne du fichier .txt dans ce dossier
        declare "$i=$value"                          # crée une variable dynamique nommée comme $i
    fi
done


# # lmax=""
lmax=$(jq -r '.lmax' galaxy_inputs/galaxy_inputs.json)

# # lmin=""   
lmin=$(jq -r '.lmin' galaxy_inputs/galaxy_inputs.json)

# max count 
maxcount=$(jq -r '.maxcount' galaxy_inputs/galaxy_inputs.json)

# min count
mincount=$(jq -r '.mincount' galaxy_inputs/galaxy_inputs.json)
# ####

### Ajout de la particule de l'argument 
if [[ -n "$seq"  && "$seq" != "null" ]]; then
    seq="-s $seq"
    echo "variable seq présente: $seq"
else 
    seq=""
fi

if [[ -n "$def"  && "$def" != "null"  ]]; then
    def="-D $def"
    echo "variable def présente : $def"
else 
    def=""
fi

if [[ -n "$ident"  && "$ident" != "null" ]]; then
    ident="-I $ident"
    echo "variable ident présente : $ident"
else 
    ident=""
fi

if [[ -n "$attrib" && "$attrib" != "null" ]]; then
    attrib="-A $attrib"
    echo "variable attrib présente : $attrib"
else 
    attrib=""
fi

if [[ -n "$hasattrib"  && "$hasattrib" != "null" ]]; then
    hasattrib="-a $hasattrib"
    echo "variable hassattribute présente : $hasattrib"
else 
    hasattrib=""
fi

if [[ -n "$predicat"  && "$predicat" != "null" ]]; then
    predicat="-p $predicat"
    echo "variable predicat présente : $predicat"
else 
    predicat=""
fi

if [[ -n "$lmax"  && "$lmax" != "null" ]]; then
    lmax="-L $lmax"
    echo "variable lmax présente : $lmax"
else 
    lmax=""
fi

if [[ -n "$lmin"  && "$lmin" != "null" ]]; then
    lmin="-l $lmin"
    echo "variable lmin présente : $lmin"
else 
    lmin=""
fi

if [[ -n "$maxcount"  && "$maxcount" != "null" ]]; then
    maxcount="-M $maxcount"
    echo "variable maxcount présente : $maxcount"
else 
    maxcount=""
fi

if [[ -n "$mincount"  && "$mincount" != "null" ]]; then
    mincount="-m $mincount"
    echo "variable mincount présente : $mincount"
else 
    mincount=""
fi

In [ ]:
# Variables Obianotate 

# VARIABLE ATTRIBUTE

## Booleen simple 
### clear

clear=$(jq -r '.clear' $json_path)

if [[ "$clear" == "true"  ]]; then
    clear="--clear"
    echo "variable clear TRUE"
else 
    clear=""
fi

In [ ]:
## Variables avec plusieurs utilisation possible -> je pense que faudrait récup dans un fichier ;))))))
deletetag=$(jq -r '.deletetag' $json_path)

if [[ -n "$deletetag" && "$deletetag" != "null" ]]; then
    IFS=' ' read -r -a tags <<< "$deletetag"
    args=()
    for tag in "${tags[@]}"; do
        args+=("--deletetag" "$tag")
    done
    deletetag="${args[@]}"
    args=""
    tags=""
else
    deletetag=""
fi


In [ ]:
 keep=$(jq -r '.keep' $json_path)

if [[ -n "$keep" && "$keep" != "null" ]]; then
    IFS=' ' read -r -a tags <<< "$keep"
    args=()
    for tag in "${tags[@]}"; do
        args+=("--keep" "$tag")
    done
    keep="${args[@]}"
    args=""
    tags=""
else
    keep=""
fi


echo $keep

-keep count
--keep count


In [ ]:
## Variables avec des caractères spéciaux (et répétitions 

listarg=(rename settag)

for i in "${listarg[@]}"; do
    if [[ -d "galaxy_inputs/$i" ]]; then
        value=$(head -n 1 "galaxy_inputs/$i"/*.txt)
        declare "$i=$value"                          
    fi
done

if [[ -n "$rename" && "$rename" != "null" ]]; then
    IFS=' ' read -r -a tags <<< "$rename"
    args=()
    for tag in "${tags[@]}"; do
        args+=("--rename" "$tag")
    done
    rename="${args[@]}"
    args=""
    tags=""
else
    rename=""
fi


if [[ -n "$settag" && "$settag" != "null" ]]; then
    IFS=' ' read -r -a tags <<< "$settag"
    args=()
    for tag in "${tags[@]}"; do
        args+=("--set-tag" "$tag")
    done
    settag="${args[@]}"
    args=""
    tags=""
else
    settag=""
fi


bash: some_command: command not found


In [26]:
# VARIABLE RELATIF A LA SEQUENCE 
corasick=$(jq -r '.corasick' $json_path)
if [[ -n "$corasick" && "$corasick" != "null" ]]; then
    corasick="--aho-corasick $corasick"
    echo "variable corasick présente : $corasick"
else 
    corasick=""
fi

length=$(jq -r '.length' $json_path)
if [[ "$length" == "true"  ]]; then
    length="--length"
    echo "variable length TRUE"
else 
    length=""
fi


pattern=$(jq -r '.pattern' $json_path)
if [[ -n "$pattern" && "$pattern" != "null" ]]; then
    pattern="--pattern $pattern"
    echo "variable pattern présente : $pattern"
else 
    pattern=""
fi

pattername=$(jq -r '.patternname' $json_path)
if [[ -n "$patternname" && "$patternname" != "null" ]]; then
    patternname="--pattern-name $patternname"
    echo "variable patternname présente : $patternname"
else 
    patternname=""
fi


In [ ]:
./obitools4/build/obiannotate $seq $def $ident $attrib $hasattrib $predicat $lmax $lmin $mincount $maxcount $clear $deletetag $keep $rename $settag $corasick $length $pattern $patternname $input_file --out $output_file

INFO[0000] Number of workers set 8                      
INFO[0000] Found 1 files to process                     
INFO[0000] /data/jwd06/pulsar_staging/86691948/inputs/dataset_692f6a32-13bd-400b-a286-6c7bce78b8a0.dat mime type: text/fasta 
INFO[0000] On output use JSON headers                   
| Reading sequences (3962/-, 8161 it/s) [0s] 


In [21]:
echo $clear $deletetag $keep $rename $settag $corasick $length $pattern $patternname $input_file --out $output_file

--keep count galaxy_inputs/inputfile/obiuniqondata24.fasta --out outputs/outputfile.fasta


In [ ]:
echo $keep